In [7]:
import numpy as np
import pandas as pd
import logging
import json
import sys
import selenium
import urllib.parse as urlparse
from operator import itemgetter
import time
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from flask import Flask, render_template, request, redirect, url_for, abort
import requests
import copy
import config
import datetime
from IPython.display import display, HTML


# Create the Flask application.
app = Flask(__name__)
app.logger.setLevel(logging.INFO)
app.config.from_object(config.ProductionConfig)
from selenium.webdriver import Chrome
import time


In [15]:
def call_espn_api(league_id):
    app.logger.info('%s - Calling ESPN API', league_id)
    url = ("https://fantasy.espn.com/apis/v3/games/fba/seasons/2020/segments/0/leagues/{}?&view=mMatchupScore&view=mPendingTransactions&view=mPositionalRatings&view=mRoster&view=mTeam&view=modular)".format(league_id))
    try:
        r = requests.get(url)
    except requests.exceptions.RequestException as ex:
        app.logger.error('%s - Could not make ESPN API call', league_id, ex)
        abort(500)
        sys.exit('Could not make ESPN API call.')
    data = r.json()
    return data

In [155]:
class Player:
    def __init__(self, name):
        self.name = name
        self.fga = 1.0
        self.fgm = 0
        self.fgp = self.fgm / self.fga
        self.fta = 1.0
        self.ftm = 0
        self.ftp = self.ftm / self.fta
        self.pts = 0
        self.ast = 0
        self.rbs = 0
        self.tov = 0
        self.m3s = 0
        self.stl = 0
        self.blk = 0
    def __str__(self):
        return self.name + " pts/reb/ast: " + format(self.pts ,'.2f') + "/" + format(self.rbs ,'.2f') + "/" + format(self.ast ,'.2f') + " stl/blk/tov: " + format(self.stl ,'.2f') + "/" + format(self.blk ,'.2f') + "/" + format(self.tov ,'.2f') + "/" + " fg%/ft%/3pm: " + format(self.fgp ,'.2f') + "/" + format(self.ftp ,'.2f') + "/" + format(self.m3s ,'.2f')

In [163]:
data["teams"]


[{'abbrev': 'SHAR',
  'currentProjectedRank': 0,
  'divisionId': 0,
  'draftDayProjectedRank': 0,
  'draftStrategy': {},
  'id': 1,
  'isActive': False,
  'location': 'The',
  'logo': 'https://g.espncdn.com/lm-static/logo-packs/core/DIS_Avengers_EndGame/DIS_Avengers_EndGame_Thanos.svg',
  'logoType': 'VECTOR',
  'nickname': 'Splash Brothers',
  'owners': ['{1E410FC9-2315-4D88-8348-CA691BE9EC38}'],
  'playoffSeed': 5,
  'points': 0.0,
  'pointsAdjusted': 0.0,
  'pointsDelta': 0.0,
  'primaryOwner': '{1E410FC9-2315-4D88-8348-CA691BE9EC38}',
  'rankCalculatedFinal': 0,
  'rankFinal': 0,
  'record': {'away': {'gamesBack': 0.0,
    'losses': 2,
    'percentage': 0.7777777777777778,
    'pointsAgainst': 0.0,
    'pointsFor': 0.0,
    'streakLength': 3,
    'streakType': 'WIN',
    'ties': 0,
    'wins': 7},
   'division': {'gamesBack': 3.0,
    'losses': 5,
    'percentage': 0.375,
    'pointsAgainst': 0.0,
    'pointsFor': 0.0,
    'streakLength': 2,
    'streakType': 'LOSS',
    'ties': 0,

In [160]:
def parse_api_to_rosters():
    data = call_espn_api(65116266)
    rosters = [[Player("") for i in range(13)] for j in range(10)]
    for i in range(10):
        for j in range(len(data["teams"][i]['roster']['entries'])):
            rosters[i][j] = Player(data["teams"][i]['roster']['entries'][j]['playerPoolEntry']['player']['fullName'])
            x = data["teams"][i]['roster']['entries'][j]['playerPoolEntry']['player']['stats']
            statIndex = 0
            for k in range(len(x)):
                if x[k]['seasonId'] == 2020 and x[k]['statSplitTypeId'] == 0 and x[k]['externalId'] == '2020' and x[k]['statSourceId'] == 0:
                    statIndex = k
                    if not x[k]['stats']:
                        for l in range(len(x)):
                            if x[l]['seasonId'] == 2019 and x[l]['statSplitTypeId'] == 0 and x[l]['externalId'] == '2019' and x[l]['statSourceId'] == 0:
                                statIndex = l
            seasonStats = x[statIndex]['averageStats']
            rosters[i][j].pts = seasonStats['0']
            rosters[i][j].fgm = seasonStats['13']
            rosters[i][j].fga = seasonStats['14']
            rosters[i][j].fgp = seasonStats['13'] / seasonStats['14']
            rosters[i][j].ftm = seasonStats['15']
            rosters[i][j].fta = seasonStats['16']
            rosters[i][j].ftp = seasonStats['15'] / seasonStats['16']
            rosters[i][j].rbs = seasonStats['6']
            rosters[i][j].ast = seasonStats['3']
            rosters[i][j].stl = seasonStats['2']
            rosters[i][j].blk = seasonStats['1']
            rosters[i][j].tov = seasonStats['11']
            rosters[i][j].m3s = seasonStats['17']
    return rosters
rosters = parse_api_to_rosters()
for roster in rosters:
    for player in roster:
        print(player)
    print()

[2020-03-21 16:15:54,920] INFO in <ipython-input-15-4304b857eff1>: 65116266 - Calling ESPN API


Nikola Jokic pts/reb/ast: 20.20/10.20/6.86 stl/blk/tov: 1.18/0.66/3.11/ fg%/ft%/3pm: 0.53/0.81/1.09
Bradley Beal pts/reb/ast: 30.54/4.25/6.09 stl/blk/tov: 1.25/0.40/3.40/ fg%/ft%/3pm: 0.46/0.84/2.98
Kristaps Porzingis pts/reb/ast: 19.24/9.47/1.75 stl/blk/tov: 0.71/2.08/1.65/ fg%/ft%/3pm: 0.42/0.78/2.47
Hassan Whiteside pts/reb/ast: 16.28/14.25/1.21 stl/blk/tov: 0.39/3.07/1.92/ fg%/ft%/3pm: 0.62/0.68/0.07
Lou Williams pts/reb/ast: 18.65/3.10/5.73 stl/blk/tov: 0.67/0.17/2.87/ fg%/ft%/3pm: 0.42/0.86/1.78
Duncan Robinson pts/reb/ast: 13.28/3.26/1.40 stl/blk/tov: 0.54/0.23/0.92/ fg%/ft%/3pm: 0.47/0.91/3.74
Stephen Curry pts/reb/ast: 20.80/5.20/6.60 stl/blk/tov: 1.00/0.40/3.20/ fg%/ft%/3pm: 0.40/1.00/2.40
Seth Curry pts/reb/ast: 12.58/2.36/2.02 stl/blk/tov: 0.61/0.14/1.02/ fg%/ft%/3pm: 0.50/0.84/2.31
Coby White pts/reb/ast: 13.22/3.54/2.69 stl/blk/tov: 0.75/0.09/1.68/ fg%/ft%/3pm: 0.39/0.79/2.05
Al Horford pts/reb/ast: 12.03/6.87/4.07 stl/blk/tov: 0.87/0.92/1.08/ fg%/ft%/3pm: 0.44/0.75/1.48


In [3]:
def parse_html_table(table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [4]:
def getHashtagTable():
    url = 'https://hashtagbasketball.com/fantasy-basketball-rankings'
    url_get = requests.get(url)
    soup = BeautifulSoup(url_get.content, 'lxml')
    table = soup.find_all('table')[2]
    table = parse_html_table(table)
    table = table.replace(r'\n',' ', regex=True)
    hashtag_table = table.replace(r'\r',' ', regex=True)
    hashtag_table.set_index("PLAYER", inplace = True)
    return hashtag_table

In [5]:
def getBBRTable():
    url = 'https://www.basketball-reference.com/leagues/NBA_2020_per_game.html'
    url_get = requests.get(url)
    soup = BeautifulSoup(url_get.content, 'lxml')
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    headers = headers[1:]
    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
    bbr_table = pd.DataFrame(player_stats, columns = headers)
    bbr_table.set_index("Player", inplace = True)
#     bbr_table.head(10)
    return bbr_table

In [6]:
def getBBMTable():
    url = 'https://basketballmonster.com/playerrankings.aspx'
    url_get = requests.get(url)
    # time.sleep(1)
    soup = BeautifulSoup(url_get.content, 'lxml')
    table = soup.find_all('table')[0]
    bbm_table = parse_html_table(table)
    bbm_table.set_index("Name", inplace = True)
    return bbm_table

In [7]:
def getRosters(leagueID):
    url = 'https://fantasy.espn.com/basketball/team?leagueId=' + str(leagueID) + '&teamId='
    # url = 'https://fantasy.espn.com/basketball/team?leagueId=65116266&teamId='
    # data = call_espn_api(65116266)
    # print(data)
    webdriver = "C:\Downloads\chromedriver_win32\chromedriver.exe"
    driver = Chrome(webdriver)
    all_teams = [[0 for i in range(13)] for j in range(10)]
    for i in range(1, 11):
        curr = url + str(i)
        driver.get(curr)
        time.sleep(4)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')[0]
        player_list = table.find_all('img')
        players = set()
        for player in player_list:
            if player.get('alt') != '':
                players.add(player.get('alt'))
        all_teams[i-1] = list(players)
    return all_teams

In [8]:
def genNameMapping(all_teams):
    name_mapping = {}
    name_mapping["Kristaps Porzingis"] = "Kristaps Porziņģis"
    name_mapping["Nikola Jokic"] = "Nikola Jokić"
    name_mapping["Nikola Vucevic"] = "Nikola Vučević"
    name_mapping["Taurean Prince"] = "Taurean Waller-Prince"
    name_mapping["Marvin Bagley III"] = "Marvin Bagley"
    name_mapping["Wendell Carter Jr."] = "Wendell Carter"
    name_mapping["Jonas Valanciunas"] = "Jonas Valančiūnas"
    name_mapping["Luka Doncic"] = "Luka Dončić"
    name_mapping["Jaren Jackson Jr."] = "Jaren Jackson"
    name_mapping["Davis Bertans"] = "Dāvis Bertāns"
    name_mapping["PJ Tucker"] = "P.J. Tucker"
    name_mapping["PJ Washington"] = "P.J. Washington"
    name_mapping["Marcus Morris Sr."] = "Marcus Morris"
    name_mapping["Tomas Satoransky"] = "Tomáš Satoranský"
    name_mapping["JJ Redick"] = "J.J. Redick"
    name_mapping["Kelly Oubre Jr."] = "Kelly Oubre"
    name_mapping["Dennis Schroder"] = "Dennis Schröder"
    name_mapping["Bojan Bogdanovic"] = "Bojan Bogdanović"
    name_mapping["Bogdan Bogdanovic"] = "Bogdan Bogdanović"
    name_mapping["Goran Dragic"] = "Goran Dragić"
    name_mapping["Gary Payton II"] = "Gary Payton"
    name_mapping["Jakob Poeltl"] = "Jakob Pöltl"
    name_mapping["Michael Porter Jr."] = "Michael Porter"
    name_mapping["Larry Nance Jr."] = "Larry Nance"
    return name_mapping
def genTeamMapping():
    team_map = {}
    team_map['janav'] = 0
    team_map['elliot'] = 1
    team_map['calvin'] = 2
    team_map['karthik'] = 3
    team_map['eric'] = 4
    team_map['garrett'] = 5
    team_map['nikhil'] = 6
    team_map['risit'] = 7
    team_map['aj'] = 8
    team_map['norman'] = 9
    return team_map

In [9]:
def addTeamData(df, label, rosters, name_mapping):
    team_data = [0 for j in range(10)]
    for i in range(0, 10):
        val = 0
        for j in range(0, len(rosters[i])):
            name = rosters[i][j]
            if name not in df.index:
                if name in name_mapping.keys():
                    name = name_mapping[name]
                if name not in df.index:
                    print("missing" + name)
                    continue
            row = df.loc[name]
            if len(row) == 3:
                row = row.loc[row['Tm'] == 'TOT']
            val += float(row[label])
        team_data[i] = val
    return team_data
def addTeamDataHashtag(df, label, rosters, name_mapping):
    team_data = [0 for j in range(10)]
    for i in range(0, 10):
        val = 0
        for j in range(0, len(rosters[i])):
            name = " " + rosters[i][j] + " "
            if name == " PJ Tucker ":
                name = " P.J. Tucker "
            elif name == " PJ Washington ":
                name = " P.J. Washington "
            if name not in df['TOTAL']:
                name = name + " "
                if name not in df['TOTAL']:
                    continue
            row = df.loc[name]
            val += float(row[label])
        team_data[i] = val
    return team_data
def rank_list(input, order):
    output = [0] * len(input)
    for i, x in enumerate(sorted(range(len(input)), key=lambda y: order *input[y])):
        output[x] = i + 1
    return output
def genRankTable(all_teams, name_mapping, hashtag_table, bbr_table, teamList):
    fga = addTeamData(bbr_table, 'FGA', all_teams, name_mapping)
    fgm = addTeamData(bbr_table, 'FG', all_teams, name_mapping)
    fg = [rank_list([fgm[i]/fga[i] for i in range(0, len(fga))], -1)]
    fta = addTeamData(bbr_table, 'FTA', all_teams, name_mapping)
    ftm = addTeamData(bbr_table, 'FT', all_teams, name_mapping)
    ft = [rank_list([ftm[i]/fta[i] for i in range(0, len(fta))], -1)]
#     points = [rank_list(addTeamDataHashtag(hashtag_table, 'PTS', all_teams, name_mapping), -1)]
#     rebounds = [rank_list(addTeamDataHashtag(hashtag_table, 'TREB', all_teams, name_mapping), -1)]
#     assists = [rank_list(addTeamDataHashtag(hashtag_table, 'AST', all_teams, name_mapping), -1)]
#     steals = [rank_list(addTeamDataHashtag(hashtag_table, 'STL', all_teams, name_mapping), -1)]
#     blocks = [rank_list(addTeamDataHashtag(hashtag_table, 'BLK', all_teams, name_mapping), -1)]
#     turnovers = [rank_list(addTeamDataHashtag(hashtag_table, 'TO', all_teams, name_mapping), 1)]
#     threes = [rank_list(addTeamDataHashtag(hashtag_table, '3PM', all_teams, name_mapping), -1)]
    points = [rank_list(addTeamData(bbr_table, 'PTS', all_teams, name_mapping), -1)]
    rebounds = [rank_list(addTeamData(bbr_table, 'TRB', all_teams, name_mapping), -1)]
    assists = [rank_list(addTeamData(bbr_table, 'AST', all_teams, name_mapping), -1)]
    steals = [rank_list(addTeamData(bbr_table, 'STL', all_teams, name_mapping), -1)]
    blocks = [rank_list(addTeamData(bbr_table, 'BLK', all_teams, name_mapping), -1)]
    turnovers = [rank_list(addTeamData(bbr_table, 'TOV', all_teams, name_mapping), 1)]
    threes = [rank_list(addTeamData(bbr_table, '3P', all_teams, name_mapping), -1)]
    # total = [rankList(addTeamDataHashtag(hashtag_table, 'TOTAL', all_teams, name_mapping), -1)]
    data = [*zip(*(fg + ft + points + rebounds + assists + steals + blocks + turnovers + threes))]
    rank_table = pd.DataFrame(data, index = teamList, columns = ['fg%', 'ft%', 'points','rebounds','assists','steals','blocks','turnovers','threes'])
    rank_table["Total"] = rank_table.sum(axis = 1, skipna = True)
    return rank_table
def genStatsTable(all_teams, name_mapping, hashtag_table, bbr_table, teamList):
    fga = addTeamData(bbr_table, 'FGA', all_teams, name_mapping)
    fgm = addTeamData(bbr_table, 'FG', all_teams, name_mapping)
    fg = [[fgm[i]/fga[i] for i in range(0, len(fga))]]
    fta = addTeamData(bbr_table, 'FTA', all_teams, name_mapping)
    ftm = addTeamData(bbr_table, 'FT', all_teams, name_mapping)
    ft = [[ftm[i]/fta[i] for i in range(0, len(fta))]]
#     points = [addTeamDataHashtag(hashtag_table, 'PTS', all_teams, name_mapping)]
#     rebounds = [addTeamDataHashtag(hashtag_table, 'TRB', all_teams, name_mapping)]
#     assists = [addTeamDataHashtag(hashtag_table, 'AST', all_teams, name_mapping)]
#     steals = [addTeamDataHashtag(hashtag_table, 'STL', all_teams, name_mapping)]
#     blocks = [addTeamDataHashtag(hashtag_table, 'BLK', all_teams, name_mapping)]
#     turnovers = [addTeamDataHashtag(hashtag_table, 'TO', all_teams, name_mapping)]
#     threes = [addTeamDataHashtag(hashtag_table, '3PM', all_teams, name_mapping)]
    points = [addTeamData(bbr_table, 'PTS', all_teams, name_mapping)]
    rebounds = [addTeamData(bbr_table, 'TRB', all_teams, name_mapping)]
    assists = [addTeamData(bbr_table, 'AST', all_teams, name_mapping)]
    steals = [addTeamData(bbr_table, 'STL', all_teams, name_mapping)]
    blocks = [addTeamData(bbr_table, 'BLK', all_teams, name_mapping)]
    turnovers = [addTeamData(bbr_table, 'TOV', all_teams, name_mapping)]
    threes = [addTeamData(bbr_table, '3P', all_teams, name_mapping)]
    # total = [rankList(addTeamDataHashtag(hashtag_table, 'TOTAL', all_teams, name_mapping), -1)]
    data = [*zip(*(fg + ft + points + rebounds + assists + steals + blocks + turnovers + threes))]
    stats_table = pd.DataFrame(data, index = teamList, columns = ['fg%', 'ft%', 'points','rebounds','assists','steals','blocks','turnovers','threes'])
    return stats_table

In [10]:
def calculate_matchups(team_name, rank_table):
    my_team = rank_table.loc[team_name]
    would_beat = []
    would_lose = []
    overall_margin = []
    for i in range(10):
        margin = []
        score = 0
        curr_team = rank_table.iloc[i]
        if curr_team.name == my_team.name:
            continue
        for j in range(9):
            if my_team.iloc[j] < curr_team.iloc[j]:
                score += 1
                margin.append(curr_team.iloc[j] - my_team.iloc[j])
        if score >= 5:
            would_beat.append(curr_team.name)
        else:
            would_lose.append(curr_team.name)
        avg = sum(margin)/len(margin)
#         print(my_team.name + " " + str(score) + " " + curr_team.name + " " + str(9 - score) + " by an average of " + str(avg))
    return would_beat, would_lose

In [11]:
def genRankList(table):
    num_beaten = [0 for i in range(0, table.shape[0])]
    teams = table.index
    matchups = [[0 for i in range(2)]for i in range(0, table.shape[0])]
    for i in range(len(teams)):
        beats, loses = calculate_matchups(teams[i],table)
        num_beaten[i] = len(beats)
        beats_str = " "
        for j in range(len(beats)):
            beats_str += beats[j] + ", "
        loses_str = " "
        for j in range(len(loses)):
            loses_str += loses[j] + ", "
#         print(teams[i] + " beats" + beats_str + "and loses to" + loses_str)
#         print("")
        matchups[i][0] = beats
        matchups[i][1] = loses
    return rank_list(num_beaten, -1), matchups


In [12]:
def printRow(df, index):
    df = {x: y for x, y in zip(df.columns, df.loc[index])}
    sf = pd.DataFrame(df, index=[index])
    display(sf)

In [13]:
def trade_analyzer(rosters, team1, team2, players_from1, players_from2, refresh, hashtagTable, bbrTable):
    if len(players_from1) != len(players_from2):
        print("The number of players exchanged must be equal")
    name_mapping = genNameMapping(rosters)
    if (refresh == 1):
        hashtagTable, bbrTable, bbmTable, rosters = refreshData(65116266)
    for player in players_from1:
        if player not in bbrTable.index:
            player = name_mapping[player]
        printRow(bbrTable, player)
    for player in players_from2:
        if player not in bbrTable.index:
            player = name_mapping[player]
        printRow(bbrTable, player)
    old_data = genRankTable(rosters, name_mapping, hashtagTable, bbrTable, team_map.keys())
    old_stats = genRankTable(rosters, name_mapping, hashtagTable, bbrTable, team_map.keys())
    old_rankList, matchups = genRankList(old_data)
    print("Prior to the trade " + team1 + " was ranked "+ str(old_rankList[team_map[team1]]))
    printRow(old_data, team1)
    print("Prior to the trade " + team2 + " was ranked "+ str(old_rankList[team_map[team2]]))
    printRow(old_data, team2)
    new_team1 = [player for player in rosters[team_map[team1]] if player not in players_from1] + players_from2
    new_team2 = [player for player in rosters[team_map[team2]] if player not in players_from2] + players_from1
    new_rosters =[[j for j in i] for i in rosters]
    new_rosters[team_map[team1]] = new_team1
    new_rosters[team_map[team2]] = new_team2
    new_data = genRankTable(new_rosters, name_mapping, hashtagTable, bbrTable, team_map.keys())
    new_rankList, matchups = genRankList(new_data)
    print("After the trade " + team1 + " was ranked "+ str(new_rankList[team_map[team1]]))
    printRow(new_data, team1)
    print("After the trade " + team2 + " was ranked "+ str(new_rankList[team_map[team2]]))
    printRow(new_data, team2)
#     waiver_analyzer(new_rosters, "eric", ["Kris Dunn"], ["Chris Boucher"], hashtagTable, bbrTable)

    
    
def waiver_analyzer(rosters, team, players_in, players_out, hashtagTable, bbrTable):
    if len(players_in) != len(players_out):
        print("The number of players exchanged must be equal")
    name_mapping = genNameMapping(rosters)
    for player in players_in:
        if player not in bbrTable.index:
            player = name_mapping[player]
        printRow(bbrTable, player)
    for player in players_out:
        if player not in bbrTable.index:
            player = name_mapping[player]
        printRow(bbrTable, player)
    team_map = genTeamMapping()
    old_data = genRankTable(rosters, name_mapping, hashtagTable, bbrTable, team_map.keys())
    old_rankList, matchups = genRankList(old_data)
    print("Prior to the pickup " + team + " was ranked "+ str(old_rankList[team_map[team]]))
    printRow(old_data, team)
    print(matchups[team_map[team]])
    new_team = [player for player in rosters[team_map[team]] if player not in players_out] + players_in
    new_rosters =[[j for j in i] for i in rosters]
    new_rosters[team_map[team]] = new_team
    new_data = genRankTable(new_rosters, name_mapping, hashtagTable, bbrTable, team_map.keys())
    new_data = genRankTable(new_rosters, name_mapping, hashtagTable, bbrTable, team_map.keys())
    new_rankList, matchups= genRankList(new_data)
    print("After the pickup " + team + " was ranked "+ str(new_rankList[team_map[team]]))
    print(matchups[team_map[team]])
    printRow(new_data, team)


In [14]:
def refreshData(leagueID):
    hashtagTable = getHashtagTable()
    bbrTable = getBBRTable()
    bbmTable = getBBMTable()
    rosters = getRosters(leagueID)
    return hashtagTable, bbrTable, bbmTable, rosters
hashtagTable, bbrTable, bbmTable, rosters = refreshData(65116266)


In [15]:
def getWorstPlayer(rankings, player_list, rosters):
    worst = ""
    worstRank = 0
    name_mapping = genNameMapping(rosters)
    for player in player_list:
        if player not in rankings.index:
            player = " " + name_mapping[player] + " "
        if int(rankings.loc[player]["R#"].strip()) > worstRank:
            worst = player
            worstRank = rankings.loc[player]["R#"]
    return worst

In [22]:
def getSoup(url,sleep_time ):
    webdriver = "C:\Downloads\chromedriver_win32\chromedriver.exe"
    driver = Chrome(webdriver)
    driver.get(url)
    time.sleep(sleep_time)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [99]:
def getGames(soup):
    player_table = soup.find("tbody", class_="Table2__tbody").find_all("tr", class_="Table2__tr")
    print(len(player_table))
    player_mapping = {}
    for i in range(0, len(player_table)):
        player_row = soup.find("tbody", class_="Table2__tbody").find_all("tr", class_="Table2__tr", attrs={"data-idx": str(i)})
        player_name = soup.find("tbody", class_="Table2__tbody").find("tr", class_="Table2__tr", attrs={"data-idx": str(i)}).find("td", class_= "Table2__td Table2__td--fixed-width", style = "width: 228px;").find("div")['title']
        for j in range(0,len(player_row)):
            todays_game = soup.find("tbody", class_="Table2__tbody").find("tr", class_="Table2__tr", attrs={"data-idx": str(i)}).find_all("td", class_= "Table2__td Table2__td--fixed-width", style = "width: 60px;")[1]
            todays_game = str(todays_game)
            todays_soup = BeautifulSoup(todays_game, 'html.parser')
            todays_game = todays_soup.find("div", class_="jsx-2810852873 table--cell opp ml4").getText()
#             print(todays_game)
            rest_of_games = soup.find("tbody", class_="Table2__tbody").find("tr", class_="Table2__tr", attrs={"data-idx": str(i)}).find_all("td", class_= "Table2__td", attrs={'style': None})
            game_text = ""
            for i in range(len(rest_of_games)):
                game_text = game_text + str(rest_of_games[i])
            rest_soup = BeautifulSoup(game_text, 'html.parser')
            rest_of_games = rest_soup.find_all(text=True)
            player_mapping[player_name] = [todays_game] + rest_of_games
    return player_mapping

def getAllMatchups(soup):
    soups = soup.find_all("div", class_ = "jsx-1241409305 matchup-score two-links")
    team_stats = {}
    matchups = {}
    for soup in soups:
        #get team id's and construct matchup mappings
        teams = soup.find_all("li", class_="ScoreboardScoreCell__Item")
        team1_link = BeautifulSoup(str(teams[0]), 'html.parser').find("a")['href']
        team1_id = team1_link[56:58]
        if team1_id != "10":
            team1_id = team1_id[:1]
        team2_link = BeautifulSoup(str(teams[1]), 'html.parser').find("a")['href']
        team2_id = team2_link[56:58]
        if team2_id != "10":
            team2_id = team2_id[:1]
        matchups[int(team1_id)] = int(team2_id)
        matchups[int(team2_id)] = int(team1_id)
        
        
        #get stats for each team and construct player to stats mapping
        stats_table = soup.find("table", class_ = "Table2__table-scroller Table2__table")
        categories = stats_table.find("thead").find_all("th", class_ ="Table2__th")
        temp = ""
        for i in range(len(categories)):
            temp = temp + str(categories[i])
        header_soup = BeautifulSoup(temp, 'html.parser')
        categories = header_soup.find_all(text=True)
        
        player1_stats = stats_table.find("tbody").find("tr", attrs={"data-idx": str(0)}).find_all(text=True)[1:]
        player1_stats = [float(i) for i in player1_stats]
        team_stats[team1_id] = player1_stats
        
        player2_stats = stats_table.find("tbody").find("tr", attrs={"data-idx": str(1)}).find_all(text=True)[1:]
        player2_stats = [float(i) for i in player2_stats]
        team_stats[team2_id] = player2_stats
    return matchups, team_stats, categories
def matchup_predictor(table, days, teamID, matchups, categories, team_stats):
    schedule_url = 'https://fantasy.espn.com/basketball/team?leagueId=65116266&teamId=' + str(teamID) + '&view=schedule'    
    soup = getSoup(schedule_url, 4)
    player_mapping1 = getGames(soup)
    player1_num_games = {}
    for player in player_mapping1.keys():
        player_mapping1[player] = player_mapping1[player][:days]
        player1_num_games[player] = sum([1 for game in player_mapping1[player] if game != "--"])
    schedule_url = 'https://fantasy.espn.com/basketball/team?leagueId=65116266&teamId=' + str(matchups[teamID]) + '&view=schedule'    
    soup = getSoup(schedule_url, 4)
    player_mapping2 = getGames(soup)
    print(player_mapping2)
    player2_num_games = {}
    for player in player_mapping2.keys():
        player_mapping2[player] = player_mapping2[player][:days]
        player2_num_games[player] = sum([1 for game in player_mapping2[player] if game != "--"])
    for player in player1_num_games.keys():
        for 
    
scoreboard_url = "https://fantasy.espn.com/basketball/league/scoreboard?leagueId=65116266"
# scoreboard_soup = getSoup(scoreboard_url, 4)
matchups, team_stats, categories = getAllMatchups(scoreboard_soup)
print(team_stats)
matchup_predictor(bbrTable, 1, 5, matchups, categories, team_stats)

{'7': [0.4548, 0.725, 33.0, 177.0, 99.0, 33.0, 10.0, 56.0, 432.0], '10': [0.4824, 0.7679, 43.0, 144.0, 96.0, 32.0, 9.0, 57.0, 457.0], '3': [0.5, 0.8915, 54.0, 161.0, 114.0, 26.0, 24.0, 62.0, 549.0], '8': [0.4432, 0.8571, 43.0, 204.0, 74.0, 24.0, 26.0, 62.0, 471.0], '9': [0.4901, 0.6887, 38.0, 189.0, 136.0, 28.0, 21.0, 77.0, 509.0], '4': [0.5249, 0.7071, 20.0, 199.0, 130.0, 38.0, 18.0, 62.0, 364.0], '1': [0.4615, 0.8468, 45.0, 218.0, 118.0, 26.0, 36.0, 63.0, 547.0], '2': [0.4614, 0.8699, 62.0, 192.0, 107.0, 25.0, 14.0, 61.0, 551.0], '5': [0.4372, 0.7303, 64.0, 145.0, 73.0, 26.0, 28.0, 48.0, 463.0], '6': [0.4516, 0.8397, 58.0, 162.0, 135.0, 41.0, 15.0, 61.0, 616.0]}
13
13
{'Derrick Rose': ['--', 'Cle', '--', '@Bkn', '--', 'Tor', '--', 'Den'], 'James Harden': [' @Den ', '@Utah', '--', '@Por', '--', 'Dal', '--', 'NO'], 'T.J. Warren': [' @Por ', '--', '--', 'Chi', '--', '--', 'NY', '--'], 'Nemanja Bjelica': ['--', '@Min', '--', 'OKC', '@LAC', '--', 'LAL', '--'], 'Nikola Vucevic': [' LAC ', 

In [15]:
name_mapping = genNameMapping(rosters)
team_map = genTeamMapping()
data = genRankTable(rosters, name_mapping, hashtagTable, bbrTable, team_map.keys())
rankList, matchups = genRankList(data)
stats_table = genStatsTable(rosters, name_mapping, hashtagTable, bbrTable, team_map.keys())
data["rank"] = rankList
display(data)
display(stats_table)

,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes,Total,rank
janav,7,3,6,7,5,10,4,3,5,50,5
elliot,4,2,3,5,4,7,10,6,3,44,3
calvin,10,1,4,9,3,4,5,7,1,44,1
karthik,1,10,10,2,8,5,3,1,10,50,6
eric,6,6,9,6,9,8,1,2,4,51,7
garrett,8,4,2,8,2,2,7,9,2,44,2
nikhil,2,9,8,1,7,6,6,5,9,53,8
risit,3,5,5,4,10,9,2,4,8,50,9
aj,9,7,7,10,1,1,9,8,7,59,10
norman,5,8,1,3,6,3,8,10,6,50,4


,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes
janav,0.462275,0.814894,216.8,75.3,46.2,10.2,10.3,25.5,23.7
elliot,0.477654,0.830357,234.2,78.8,47.2,11.5,5.7,27.2,25.9
calvin,0.448652,0.854043,226.1,64.1,48.9,12.6,9.4,28.6,26.6
karthik,0.528548,0.680653,166.1,90.6,43.4,12.4,10.8,23.4,7.5
eric,0.464923,0.800959,203.3,75.9,43.3,11.4,11.5,23.9,24.3
garrett,0.454822,0.812854,248.7,70.3,57.0,14.1,7.1,30.9,26.3
nikhil,0.496412,0.743219,208.2,97.1,44.0,12.1,9.1,26.7,15.0
risit,0.489374,0.803063,219.4,89.2,37.6,11.3,10.9,25.5,16.7
aj,0.454650,0.785047,210.1,63.6,66.2,14.7,6.0,30.0,19.4
norman,0.469697,0.768072,257.6,89.7,45.9,13.5,6.8,32.0,21.2


In [22]:
# waiver_analyzer(rosters, "eric", ["PJ Washington"], ["Jeremy Lamb"], hashtagTable, bbrTable)
trade_analyzer(rosters, "garrett", "calvin", ['Fred VanVleet', 'D\'Angelo Russell'],["John Collins", "Shai Gilgeous-Alexander"], 0, hashtagTable, bbrTable)

,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Fred VanVleet,PG,25,TOR,48,48,35.8,6.0,14.6,.409,2.7,...,.843,0.4,3.4,3.8,6.6,1.9,0.3,2.3,2.5,17.6


,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
D'Angelo Russell,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS


,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
John Collins,PF,22,ATL,33,33,32.5,8.4,14.6,.577,1.3,...,.800,2.7,7.6,10.3,1.4,0.7,1.6,1.8,3.5,20.8


,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Shai Gilgeous-Alexander,SG,21,OKC,58,58,35.5,7.0,15.0,.464,1.2,...,.809,0.8,5.6,6.3,3.3,1.1,0.7,1.9,1.7,19.4


Prior to the trade garrett was ranked 2


...

,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes,Total
calvin,10,1,4,9,3,4,5,7,1,44


After the trade garrett was ranked 1


,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes,Total
garrett,5,4,2,5,3,4,6,7,5,41


After the trade calvin was ranked 2


,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes,Total
calvin,10,1,4,10,2,2,7,9,1,46


In [16]:
# rosters[team_map["janav"]][7] = "Kendrick Nunn"
rosters[team_map["eric"]]

['Davis Bertans',
 'Jaren Jackson Jr.',
 'Zach LaVine',
 'Gorgui Dieng',
 'Tobias Harris',
 "Devonte' Graham",
 'Nerlens Noel',
 'Maxi Kleber',
 'Luka Doncic',
 'Malcolm Brogdon',
 'Richaun Holmes',
 'Will Barton',
 'Myles Turner']

In [21]:
matchups[team_map["eric"]]

[['janav', 'risit', 'aj'],
 ['elliot', 'calvin', 'karthik', 'garrett', 'nikhil', 'norman']]

In [49]:
bbrTable

,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Player,,,,,,,,,,,,,,,,,,,,,
Steven Adams,C,26,OKC,36,36,28.3,5.0,8.2,.614,0.0,...,.565,3.6,6.5,10.2,2.7,0.7,1.1,1.5,1.9,12.0
Bam Adebayo,C,22,MIA,39,39,34.3,6.1,10.2,.593,0.0,...,.684,2.6,7.8,10.4,4.5,1.3,1.2,2.8,2.6,15.8
LaMarcus Aldridge,C,34,SAS,36,36,33.0,7.5,14.7,.508,1.2,...,.836,2.0,5.6,7.6,2.5,0.6,1.8,1.4,2.3,18.9
Nickeil Alexander-Walker,SG,21,NOP,31,0,13.6,2.1,6.2,.342,1.2,...,.571,0.2,2.0,2.2,2.0,0.3,0.1,1.0,1.2,5.8
Grayson Allen,SG,24,MEM,25,0,16.9,2.7,6.1,.441,1.3,...,.897,0.2,2.1,2.3,1.3,0.3,0.0,0.9,1.3,7.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Trae Young,PG,21,ATL,36,36,35.1,9.3,21.0,.443,3.4,...,.846,0.6,4.0,4.5,8.4,1.2,0.1,4.8,1.6,28.9
Cody Zeller,C,27,CHO,39,20,23.1,4.5,8.4,.537,0.4,...,.704,2.9,4.3,7.2,1.3,0.6,0.4,1.4,2.5,11.7
NaN,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
